### 3DCT Refactor

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# markers_3d: correlation points in 3D image (FM)
# markers_2d: correlation points in 2D image (FIB/SEM)
# spots_3d: points of interest in 3D image (FM)
# rotation_center: rotation center of the 3D image (FM)
    ## The default value is set as the center of a cube with an edge length equal to the longest edge of the image volume
# results_file: path to save the results (.txt)
# imageProps: dictionary with image properties (voxel size, image size, etc.)

# installer napari: req 
# #conda install -c conda-forge libstdcxx-ng
# pip install -e . 


from tdct import correlation
from pprint import pprint
import os
import csv
import tifffile as tff
import numpy as np

PATH = "/home/patrick/development/data/CORRELATION/3dct/3D_correlation_test_dataset"
PATH = "/home/patrick/github/3DCT/3D_correlation_test_dataset"


fib_coord_filename = os.path.join(PATH, "IB_coordinates.txt")
fm_coord_filename = os.path.join(PATH, "LM_coordinates_withPOI.txt")

fib_image_filename = os.path.join(PATH, "IB_image.tif")
fm_image_filename = os.path.join(PATH, "LM_image_stack_reslized.tif")

def parse_coordinate_file(filename: str, delimiter: str = "\t") -> list:
    coords: list = []
    with open(filename) as csv_file:
        for row in csv.reader(csv_file, delimiter=delimiter):
            coords.append([field for field in row])
    return coords

fib_coordinates = parse_coordinate_file(fib_coord_filename)
fm_coordinates = parse_coordinate_file(fm_coord_filename)

fib_coordinates = np.array(fib_coordinates, dtype=np.float32)
fm_coordinates = np.array(fm_coordinates, dtype=np.float32)

print(f"FIB Coordinates:")
pprint(fib_coordinates)
print(f"FM Coordinates:")
pprint(fm_coordinates)

In [ ]:
# extract the number of coordinates for each type (fib, fm, poi)

nrows_fib = len(fib_coordinates)
nrows_fm = len(fm_coordinates)
nrows_poi = nrows_fm - nrows_fib

print(f"Number of FIB coordinates: {nrows_fib}")
print(f"Number of FM coordinates: {nrows_fm}")
print(f"Number of POI coordinates: {nrows_poi}")

# points of interest are excess coordinates in the FM coordinate file
fib_coordinates_corr = fib_coordinates
fm_coordinates_corr = fm_coordinates[:nrows_fib]
poi_coordinates = fm_coordinates[nrows_fib:]

print(fib_coordinates_corr)
print(fm_coordinates_corr)
print(poi_coordinates)

# points are selected in the image coordinates (0, 0 top left)
# z level is in pixels as well
# no need to convert to physical coordinates

In [ ]:
# rotation center is the center of the image volume

fm_image = tff.imread(fm_image_filename)
fib_image = tff.imread(fib_image_filename)

print(fm_image.shape)
print(fib_image.shape)

halfmax_dim = max(fm_image.shape) * 0.5
rotation_center = (halfmax_dim, halfmax_dim, halfmax_dim)
print(rotation_center)


In [ ]:
# extract fib pixel size from tiff metadata
# open tiff file
with tff.TiffReader(fib_image_filename) as tif:
    for page in tif.pages:
        for tag in page.tags.values():
            if tag.name == 'FEI_HELIOS':
                md = tag.value

pprint(md["Scan"]["PixelWidth"])    
pixel_size = md["Scan"]["PixelWidth"]


In [ ]:
# image properties: used for converting coordinates to physical units (microscope coordinates)

print(f"FM Shape: {fm_image.shape}")
print(f"Initial FIB Shape: {fib_image.shape}")

if fib_image.ndim == 3:
    fib_shape = fib_image.shape[:-1]
    print(f"2D FIB Shape: {fib_shape}")

# standard resolutions: (old), needs to be modernised
standard_res = [(442, 512), (884, 1024), (1768, 2048), (3536, 4096), (7072, 8192)]

# find closest resolution
res = min(standard_res, key=lambda x: abs(x[0] - fib_shape[0]))

if fib_shape != res:
    print(f"Clipped to closest standard resolution: {res}")

# fib image shape minus metadata, fib_pixelsize (microns), fm_image_shape
image_props = [res, pixel_size*1e6, fm_image.shape]
print(f"Image Properties: {image_props}")

In [ ]:
fib_coordinates_corr[:, :2]

In [ ]:
import datetime

timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
results_file = os.path.join(PATH, f"{timestamp}_correlation.txt")

cor_ret = ret = correlation.main(markers_3d=fm_coordinates_corr,
                                markers_2d=fib_coordinates_corr,
                                spots_3d=poi_coordinates,
                                rotation_center=rotation_center,
                                results_file=results_file,
                                imageProps=image_props
                                )

In [ ]:
# transf, transf3d, spots2d, deltad2d, cm_3d_markers, mod_trans

from tdct.app import save_correlation_data, parse_correlation_result

# input data
input_data = {
    "fib_coordinates": fib_coordinates_corr.tolist(),
    "fm_coordinates": fm_coordinates_corr.tolist(),
    "poi_coordinates": poi_coordinates.tolist(),
    "image_properties": {
        "fib_image_filename": fib_image_filename,
        "fib_image_shape": list(image_props[0]),
        "fib_pixel_size_um": float(image_props[1]),
        "fm_image_filename": fm_image_filename,
        "fm_image_shape": list(fm_image.shape),
    },
    "rotation_center": list(rotation_center),
    "rotation_center_custom": list(rotation_center)
}
print(input_data["image_properties"]["fib_image_shape"])
# output data
correlation_data = parse_correlation_result(cor_ret, input_data)

# full correlation data
full_correlation_data = {
    "metadata": {
        "timestamp": datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'),
        "data_path": PATH,
        "csv_path": "", 
        "project_path": "",
    },
    "correlation": correlation_data,

}
save_correlation_data(full_correlation_data, PATH)


In [ ]:
# cor_ret[0] # transformation matrix
# cor_ret[1] # reprojected 3D points
# cor_ret[2] # spots2d
# cor_ret[3] # difference reprojected 3D points and 2D points
# cor_ret[4] # cm_3d_markers
# cor_ret[5] # translation around rotation center (custom)

In [ ]:
print(cor_ret[1].T)

print(fib_coordinates_corr[:, :2] + cor_ret[3].T)

# convert from q list to matrix
# q = np.array(full_correlation_data["rotation_quaternion"])


In [ ]:
error = full_correlation_data["correlation"]["output"]["error"]

delta_2d = np.array(error["delta_2d"]).T

print(delta_2d.shape)
# loop through all points and and print (x, y)
for dx, dy in delta_2d:
    
    # fmt as .2f str
    dx, dy = f"{dx:.2f}", f"{dy:.2f}"

    print(f"({dx}, {dy})")

In [ ]:
# pixel_size = tif.pages[0].tags['FEI_HELIOS'].value['Scan']['PixelWidth']
%load_ext autoreload
%autoreload 2

import tifffile as tff
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
FIB_IMAGE_PATH = "3D_correlation_test_dataset/IB_image.tif"


# things to consider: 
# extract pixel size
# convert to grayscale (if rgb)
# trim metadata bar
import logging

logging.basicConfig(level=logging.INFO)
from tdct.app import load_and_parse_fib_image

fib_image, pixel_size = load_and_parse_fib_image(FIB_IMAGE_PATH)

plt.imshow(fib_image, cmap='gray')
plt.show()

print(f"Pixel Size: {pixel_size*1e6}")

# fig, ax = plt.subplots(1, 3, figsize=(15, 3))

# plt.suptitle(f"Image: {os.path.basename(FIB_IMAGE_PATH)}, Pixel Size: {pixel_size*1e9:.2f} nm")

# ax[0].imshow(fib_image, cmap='gray')
# ax[0].set_title(f"Original: {fib_image.shape}")
# ax[1].imshow(img, cmap='gray')
# ax[1].set_title(f"Base: {img.shape}")
# ax[2].imshow(trimmed_img, cmap='gray')
# ax[2].set_title(f"Trimmed: {trimmed_img.shape}")
# plt.show()



### Refactor Testing


In [7]:
%load_ext autoreload
%autoreload 2

import os
import tifffile as tff
import matplotlib.pyplot as plt
import datetime
import numpy as np
from tdct import correlation
from pprint import pprint
from tdct.app import load_and_parse_fib_image, parse_coordinates


PATH = "/home/patrick/github/3DCT/3D_correlation_test_dataset"


fib_coord_filename = os.path.join(PATH, "IB_coordinates.txt")
fm_coord_filename = os.path.join(PATH, "LM_coordinates_withPOI.txt")

fib_image_filename = os.path.join(PATH, "IB_image.tif")
fm_image_filename = os.path.join(PATH, "LM_image_stack_reslized.tif")


fib_coordinates, fm_coordinates = parse_coordinates(fib_coord_filename, fm_coord_filename)

# extract the number of coordinates for each type (fib, fm, poi)

nrows_fib = len(fib_coordinates)
nrows_fm = len(fm_coordinates)
nrows_poi = nrows_fm - nrows_fib

print(f"Number of FIB coordinates: {nrows_fib}")
print(f"Number of FM coordinates: {nrows_fm}")
print(f"Number of POI coordinates: {nrows_poi}")

# points of interest are excess coordinates in the FM coordinate file
fib_coordinates = fib_coordinates
poi_coordinates = fm_coordinates[nrows_fib:]
fm_coordinates = fm_coordinates[:nrows_fib]



print("-"*50)
print(f"FIB Coordinates:")
pprint(fib_coordinates)
print(f"FM Coordinates:")
pprint(fm_coordinates)
print(f"POI Coordinates:")
pprint(poi_coordinates)

fib_image, pixel_size = load_and_parse_fib_image(fib_image_filename)
fm_image = tff.imread(fm_image_filename)

print("-"*50)
print(f"FIB Image Shape: {fib_image.shape}")
print(f"Pixel Size: {pixel_size*1e6}um")
print(f"FM Shape: {fm_image.shape}")

print("-"*50)

# rotation center
halfmax_dim = max(fm_image.shape) * 0.5
rotation_center = (halfmax_dim, halfmax_dim, halfmax_dim)
print(f"Rotation Center: {rotation_center}")


# set image properties
image_props = [fib_image.shape, pixel_size*1e6, fm_image.shape]
print(f"Image Properties: {image_props}")

print("-"*50)

timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
results_file = os.path.join(PATH, f"{timestamp}_correlation.txt")
print(f"Results File: {results_file}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Number of FIB coordinates: 9
Number of FM coordinates: 12
Number of POI coordinates: 3
--------------------------------------------------
FIB Coordinates:
array([[2.211670e+02, 3.815000e+02, 1.000000e+00],
       [6.926670e+02, 3.136670e+02, 1.000000e+00],
       [6.915000e+02, 3.175000e+02, 1.000000e+00],
       [2.636670e+02, 4.066670e+02, 1.000000e+00],
       [7.761670e+02, 3.276670e+02, 1.000000e+00],
       [3.885000e+02, 4.455000e+02, 1.000000e+00],
       [9.481670e+02, 4.473330e+02, 1.000000e+00],
       [1.015667e+03, 3.890000e+02, 1.000000e+00],
       [1.004167e+03, 4.125000e+02, 1.000000e+00]], dtype=float32)
FM Coordinates:
array([[ 354.862,  766.661,   18.314],
       [ 630.553,  137.518,   22.579],
       [ 642.876,  162.691,   19.006],
       [ 455.053,  848.229,   21.371],
       [ 744.934,  145.989,   24.275],
       [ 663.056,  931.327,   28.423],
       [1196.78 ,  563.697,   38

INFO:root:Image shape (941, 1024) does not match metadata shape (884, 1024), likely a metadata bar present
INFO:root:Cropped Shape: (884, 1024), Trimmed Shape: (884, 1024)
INFO:root:Image trimmed from (941, 1024) to (884, 1024)


--------------------------------------------------
FIB Image Shape: (884, 1024)
Pixel Size: 0.18540500000000001um
FM Shape: (107, 1253, 1540)
--------------------------------------------------
Rotation Center: (770.0, 770.0, 770.0)
Image Properties: [(884, 1024), 0.18540500000000001, (107, 1253, 1540)]
--------------------------------------------------
Results File: /home/patrick/github/3DCT/3D_correlation_test_dataset/2024-09-11_14-31-51_correlation.txt


In [79]:
cor_ret = correlation.main(markers_3d=fm_coordinates,
                                markers_2d=fib_coordinates,
                                spots_3d=poi_coordinates,
                                rotation_center=rotation_center,
                                results_file=results_file,
                                imageProps=image_props
                                )

cor_ret_2 = correlation.correlate(markers_3d=fm_coordinates,
                                markers_2d=fib_coordinates,
                                poi_3d=poi_coordinates,
                                rotation_center=rotation_center,
                                imageProps=image_props
                                )

correlation.save_results(cor_ret_2, results_file)

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: 988447.1576780467
            Iterations: 6
            Function evaluations: 7
            Gradient evaluations: 2


Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.990073435707018
            Iterations: 10
            Function evaluations: 26
            Gradient evaluations: 10
Optimization terminated successfully    (Exit mode 0)
            Current function value: 510056.95605642395
            Iterations: 5
            Function evaluations: 1
            Gradient evaluations: 1
Optimization terminated successfully    (Exit mode 0)
            Current function value: 146343.57759420108
            Iterations: 5
            Function evaluations: 1
            Gradient evaluations: 1
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.990073527675122
            Iterations: 14
            Function evaluations: 32
            Gradient evaluations: 13
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2186984.7044232762
            Iterations: 5
            Function evaluations:

In [80]:
#pre_cor_ret = cor_ret

print("-"*50)

poi2d = cor_ret[2].T
poi3d = cor_ret[1].T
d2d = cor_ret[3].T
mod_trans = cor_ret[5]

cor_ret_2 = cor_ret_2["output"]
poi2d_2 = cor_ret_2["reprojected_2d_poi"].T
poi3d_2 = cor_ret_2["reprojected_3d_coordinates"].T
d2d_2 = cor_ret_2["reprojection_error"].T
mod_trans_2 = cor_ret_2["modified_translation"]


# assert the arrays are equal
# np.testing.assert_array_equal(poi2d, poi2d_2)
np.testing.assert_array_equal(poi3d, poi3d_2)
assert np.allclose(d2d, d2d_2)
assert np.allclose(mod_trans, mod_trans_2)

print("Arrays are equal")

--------------------------------------------------
Arrays are equal


[[ 354.862  630.553  642.876  455.053  744.934  663.056 1196.78  1095.805
  1148.2  ]
 [ 766.661  137.518  162.691  848.229  145.989  931.327  563.697  214.095
   336.959]
 [  18.314   22.579   19.006   21.371   24.275   28.423   38.288   45.455
    44.255]]
